In [ ]:
cd ../../..

In [ ]:
%load_ext dotenv
%dotenv -o config/secrets/.env.linkedin.local

In [ ]:
from linkedin_easy_apply_bot.backend import config
from linkedin_easy_apply_bot.backend.main import EasyApplyBot

In [ ]:

assert config.LINKEDIN_USERNAME is not None
assert config.LINKEDIN_PASSWORD is not None

In [ ]:
from linkedin_easy_apply_bot.backend.utils import yaml_utils
parameters = yaml_utils.read_file('src/config.yaml')
parameters

In [ ]:
import time
import random
from linkedin_easy_apply_bot.backend.utils import logging_utils


log = logging_utils.get_logger(__name__)

def start_apply(positions, locations) -> None:
        start: float = time.time()
        combos: list = []
        while len(combos) < len(positions) * len(locations):
            position = positions[random.randint(0, len(positions) - 1)]
            location = locations[random.randint(0, len(locations) - 1)]
            combo: tuple = (position, location)
            if combo not in combos:
                combos.append(combo)
                log.info(f"Applying to {position}: {location}")
                location = "&location=" + location
                yield (position, location)
            if len(combos) > 500:
                break

In [ ]:
position_location_pairs = list(start_apply(parameters["positions"], parameters["locations"]))
position_location_pairs

In [ ]:
def applications_loop(bot, position, location):

        count_application = 0
        count_job = 0
        jobs_per_page = 0
        start_time: float = time.time()

        log.info("Looking for jobs.. Please wait..")
        bot.browser.set_window_position(1, 1)
        bot.browser.maximize_window()
        bot.browser, _ = bot.next_jobs_page(
            position, location, jobs_per_page, experience_level=bot.experience_level
        )
        log.info("Looking for jobs.. Please wait..")

        while time.time() - start_time < bot.MAX_SEARCH_TIME:
            try:
                log.info(
                    f"{(bot.MAX_SEARCH_TIME - (time.time() - start_time)) // 60} minutes left in this search"
                )

                # sleep to make sure everything loads, add random to make us look human.
                randoTime: float = random.uniform(1.5, 2.9)
                log.debug(f"Sleeping for {round(randoTime, 1)}")
                # time.sleep(randoTime)
                bot.load_page(sleep=0.5)

                # LinkedIn displays the search results in a scrollable <div> on the left side, we have to scroll to its bottom

                # scroll to bottom

                if bot.is_present(bot.locator["search"]):
                    scrollresults = bot.get_elements("search")
                    #     bot.browser.find_element(By.CLASS_NAME,
                    #     "jobs-search-results-list"
                    # )
                    # Selenium only detects visible elements; if we scroll to the bottom too fast, only 8-9 results will be loaded into IDs list
                    for i in range(300, 3000, 100):
                        bot.browser.execute_script(
                            "arguments[0].scrollTo(0, {})".format(i), scrollresults[0]
                        )
                    scrollresults = bot.get_elements("search")
                    # time.sleep(1)

                # get job links, (the following are actually the job card objects)
                if bot.is_present(bot.locator["links"]):
                    links = bot.get_elements("links")
                    # links = bot.browser.find_elements("xpath",
                    #     '//div[@data-job-id]'
                    # )

                    jobIDs = {}  # {Job id: processed_status}

                    # children selector is the container of the job cards on the left
                    for link in links:
                        if "Applied" not in link.text:  # checking if applied already
                            if (
                                link.text not in bot.blacklist
                            ):  # checking if blacklisted
                                jobID = link.get_attribute("data-job-id")
                                if jobID == "search":
                                    log.debug(
                                        "Job ID not found, search keyword found instead? {}".format(
                                            link.text
                                        )
                                    )
                                    continue
                                else:
                                    jobIDs[jobID] = "To be processed"
                    # if len(jobIDs) > 0:
                    #     bot.apply_loop(jobIDs)
                    bot.browser, jobs_per_page = bot.next_jobs_page(
                        position,
                        location,
                        jobs_per_page,
                        experience_level=bot.experience_level,
                    )
                else:
                    bot.browser, jobs_per_page = bot.next_jobs_page(
                        position,
                        location,
                        jobs_per_page,
                        experience_level=bot.experience_level,
                    )

            except Exception as e:
                print(e)


In [ ]:
bot = EasyApplyBot(
    username=config.LINKEDIN_USERNAME,
    password=config.LINKEDIN_PASSWORD
)


for position,location in position_location_pairs:
    applications_loop(bot, position, location)
